In [4]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from scripts.indicateur import get_typologie_rotation_CAN_synthetise as fnc
from scripts.indicateur import get_typologie_culture_CAN as typoC


ModuleNotFoundError: No module named 'scripts'

In [3]:
# ------------------ #
# IMPORT DES DONNÉES #
# ------------------ #
ENTREPOT_PATH = '~/Bureau/Datagrosyst/data_entrepot_&_outils/'
EXTERN_DATA_PATH = '~/Bureau/Datagrosyst/data_entrepot_&_outils/referentiels/'
df = {}

def import_df(df_name, path_data, sep, index_col=None):
    df[df_name] = pd.read_csv(path_data+df_name+'.csv', sep = sep, index_col=index_col, low_memory=False).replace({'\r\n': '\n'}, regex=True)

def import_dfs(df_names, path_data, sep = ',', index_col=None, verbose=False):
    for df_name in tqdm(df_names) : 
        if(verbose) :
            print(' - ', df_name)
        import_df(df_name, path_data, sep, index_col=index_col)

tables_entrepot = [
    'connection_synthetise', 'noeuds_synthetise'
]

outils_data = [
    'surface_connection_synthetise'
]

data_pour_typologie_culutre = [
    'espece_vCAN', 'culture', 'composant_culture'
]

extern_data_pour_typologie_culutre = [
    'typo_especes_typo_culture', 'typo_especes_typo_culture_marai'
]

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(tables_entrepot, ENTREPOT_PATH, sep = ',', index_col='id', verbose=False)

# import des données de l'entrepôt avec la colonne 'id' en index 
import_dfs(outils_data, ENTREPOT_PATH, sep = ',', index_col='id', verbose=False)

# import des données pour 'typologie_can_culture'
import_dfs(data_pour_typologie_culutre, ENTREPOT_PATH, sep = ',', index_col='id', verbose=False)
import_dfs(extern_data_pour_typologie_culutre, EXTERN_DATA_PATH, sep = ',', verbose=False)
df['typologie_can_culture'] = typoC(df)


NameError: name 'tqdm' is not defined

In [3]:
result = fnc(df)

In [16]:
list_culture_id = [
'fr.inra.agrosyst.api.entities.CroppingPlanEntry_60cc91e5-d085-40c7-a3dc-fc00684b05ad',
'fr.inra.agrosyst.api.entities.CroppingPlanEntry_3478eb36-740a-40b5-96fa-28d29d931605',
'fr.inra.agrosyst.api.entities.CroppingPlanEntry_94c605bb-a2c5-4f47-833e-c126edc00594',
'fr.inra.agrosyst.api.entities.CroppingPlanEntry_931d7b38-1df6-4e38-bbf5-9661ddb0a81b',
'fr.inra.agrosyst.api.entities.CroppingPlanEntry_f144f317-25b5-403d-81ca-56ed7809c705',
'fr.inra.agrosyst.api.entities.CroppingPlanEntry_1bb9ef49-7f10-4ce9-87d0-22b36cfe883a',
'fr.inra.agrosyst.api.entities.CroppingPlanEntry_48127946-206c-40f4-b69a-206c28019190'
]

vu = result.loc[result.index.isin(list_culture_id)]

cult = df['culture']
comp = df['composant_culture']

# Pour les datas
PATH = '/home/administrateur/Bureau/Datagrosyst/catalogue_script_agrosyst/02_outils/tests/data/test_get_typologie_culture_CAN/'
# comp.loc[comp.culture_id.isin(list_culture_id)].to_csv(PATH+'composant_culture'+'.csv')
# cult.loc[cult.index.isin(list_culture_id)].to_csv(PATH+'culture'+'.csv')


In [ ]:
# Pour les metadonnes
meta = pd.melt(vu.reset_index(), id_vars='culture_id', value_vars=['typocan_espece', 'typocan_espece_maraich', 'nb_espece',
       'nb_typocan_esp', 'nb_typocan_esp_maraich', 'type', 'typocan_culture','typocan_culture_maraich'])
meta = meta.set_index('culture_id')
# meta.to_csv(EXTERN_DATA_PATH + 'meta_TU_todo.csv')